<font size="8"> **Adding environmental data from available observations to unique crabeater observations** </font>  
In this notebook, we will extract environmental data from sea ice and sea surface temperature observations and add it to our data frame containing unique crabeater sightings per month and grid cell (see `04_Creating_bio_data_masks.ipynb` for more information.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [148]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42771,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:40835,Total threads: 2
Dashboard: /proxy/43123/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:36603,


# Loading unique crabeater seal observations data frame

In [4]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Cleaned_Data/unique_crabeater_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence
0,2019-03-09T00:00:00Z,2019-03,2019,3,-64.888,-62.85,-64.910,-62.9,autumn,in-between,2010,East Pacific,Antarctic,1
1,2019-03-13T00:00:00Z,2019-03,2019,3,-65.142,-64.15,-65.121,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
2,2019-03-15T00:00:00Z,2019-03,2019,3,-65.142,-64.05,-65.121,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
3,2019-03-15T00:00:00Z,2019-03,2019,3,-65.100,-63.95,-65.079,-64.0,autumn,in-between,2010,East Pacific,Antarctic,1
4,2019-03-14T00:00:00Z,2019-03,2019,3,-65.353,-64.15,-65.332,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,2006-02-19T13:00:00Z,2006-02,2006,2,-67.085,69.95,-67.064,70.0,summer,moulting,2000,Central Indian,Antarctic,1
3976,2006-01-30T13:00:00Z,2006-01,2006,1,-67.972,40.75,-67.951,40.7,summer,moulting,2000,Central Indian,Antarctic,1
3977,2006-02-19T13:00:00Z,2006-02,2006,2,-67.085,69.95,-67.106,70.0,summer,moulting,2000,Central Indian,Antarctic,1
3978,2006-02-19T13:00:00Z,2006-02,2006,2,-67.127,69.95,-67.149,70.0,summer,moulting,2000,Central Indian,Antarctic,1


# Loading crabeater seal mask

In [5]:
mask_all = xr.open_dataarray('/g/data/v45/la6889/Chapter2_Crabeaters/mask_crabeater_obs_ocean_grid.nc')

# Adding values for static variables only
Static variables referred to any physical variables that do not change over time (at least not during the time period of our interest). Examples include depth of the water column and distance to coastline. Given that we only have one value for these variables, the process of extracting data is relatively simple. We do not need to take into account the date observations were collected.

## Defining dictionary with information about static variables
This dictionary contains the column labels for each and the name of the files for each static variable to be included in our analysis. We will also define a variable containing the full path to the folder where all static variables are stored.

In [6]:
#Full path to static variables
base_dir_static = '/g/data/v45/la6889/Chapter2_Crabeaters/Static_Variables/'

#List of static variables
varDict = {'bottom_slope_deg': 'bathy_slope_GEBCO_2D.nc',
           'dist_shelf_km': 'distance_shelf.nc',
           'dist_coast_km': 'distance_coastline.nc',
           'depth_m': 'bathy_GEBCO_2D.nc'}

## Extracting data for each observation and adding it to a new column in crabeater data

In [7]:
#Looping through dictionary keys
for var in varDict:
    #Creating full path to file of interest
    file_path = os.path.join(base_dir_static, varDict[var])
    #Load as raster
    ras = xr.open_dataarray(file_path)
    #Applying mask
    ras_masked = ras*mask_all
    #Transforming masked array into data frame
    ras_df = ras_masked.to_series().dropna().reset_index()
    #Rounding up coordinate values
    ras_df = ras_df.round({'yt_ocean': 3, 'xt_ocean': 3})
    #Renaming masked data before merging to observations
    ras_df.rename(columns = {0: var}, inplace = True)
    #Adding to crabeater observations data frame
    crabeaters = crabeaters.merge(ras_df, on = ['yt_ocean', 'xt_ocean'], how = 'left').sort_values(['yt_ocean', 'xt_ocean'])
    
#Checking results
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,2004,2,-77.687,166.35,-77.708,166.3,summer,moulting,2000,West Pacific,Antarctic,1,89.980469,606.013921,32.155391,176.033340
1,2014-01-13T14:51:22Z,2014-01,2014,1,-76.842,-36.05,-76.864,-36.1,summer,moulting,2010,Atlantic,Antarctic,1,89.729942,383.202891,41.998688,1064.320435
2,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-157.25,-76.779,-157.3,summer,moulting,2000,West Pacific,Antarctic,1,89.943176,113.770361,4.724272,612.181458
3,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-156.85,-76.779,-156.8,summer,moulting,2000,West Pacific,Antarctic,1,89.897667,113.278688,2.641858,656.099976
4,2020-02-02T00:00:00Z,2020-02,2020,2,-76.378,163.95,-76.357,163.9,summer,moulting,2020,West Pacific,Antarctic,1,89.871246,603.310632,21.026712,519.266663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,1989-11-05T10:00:00Z,1989-11,1989,11,-60.047,70.25,-60.072,70.2,autumn,weaning,1980,Central Indian,Subantarctic,1,89.978432,478.561536,622.374268,4235.086426
3976,2008-12-26T00:00:00Z,2008-12,2008,12,-59.645,9.15,-59.620,9.1,summer,weaning,2000,Atlantic,Antarctic,1,89.958000,844.112672,926.684712,5462.944336
3977,1989-11-05T02:00:00Z,1989-11,1989,11,-59.493,71.85,-59.468,71.9,autumn,weaning,1980,Central Indian,Subantarctic,1,89.864761,531.076147,692.923480,4498.345215
3978,2013-11-12T00:00:00Z,2013-11,2013,11,-57.829,-26.35,-57.803,-26.3,autumn,weaning,2010,Atlantic,Antarctic,1,NaN,1443.071239,1067.076760,506.524384


## Saving data frame with static variables
Given that the dynamic variables take some time to extract. We will save intermediary results to avoid having to extract them again.

In [8]:
#Defining output folder
folder_out = '../Cleaned_Data/Env_obs'
#Checking folder exists
os.makedirs(folder_out, exist_ok = True)

#Saving file
crabeaters.to_csv(os.path.join(folder_out, 'unique_crabeater_obs_static_env.csv'), index = False)

# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [9]:
crabeaters = pd.read_csv(os.path.join(folder_out, 'unique_crabeater_obs_static_env.csv'))
#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,2004,2,-77.687,166.35,-77.708,166.3,summer,moulting,2000,West Pacific,Antarctic,1,89.980469,606.013921,32.155391,176.033340
1,2014-01-13T14:51:22Z,2014-01,2014,1,-76.842,-36.05,-76.864,-36.1,summer,moulting,2010,Atlantic,Antarctic,1,89.729942,383.202891,41.998688,1064.320435
2,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-157.25,-76.779,-157.3,summer,moulting,2000,West Pacific,Antarctic,1,89.943176,113.770361,4.724272,612.181458
3,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-156.85,-76.779,-156.8,summer,moulting,2000,West Pacific,Antarctic,1,89.897667,113.278688,2.641858,656.099976
4,2020-02-02T00:00:00Z,2020-02,2020,2,-76.378,163.95,-76.357,163.9,summer,moulting,2020,West Pacific,Antarctic,1,89.871246,603.310632,21.026712,519.266663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,1989-11-05T10:00:00Z,1989-11,1989,11,-60.047,70.25,-60.072,70.2,autumn,weaning,1980,Central Indian,Subantarctic,1,89.978432,478.561536,622.374268,4235.086426
3976,2008-12-26T00:00:00Z,2008-12,2008,12,-59.645,9.15,-59.620,9.1,summer,weaning,2000,Atlantic,Antarctic,1,89.958000,844.112672,926.684712,5462.944336
3977,1989-11-05T02:00:00Z,1989-11,1989,11,-59.493,71.85,-59.468,71.9,autumn,weaning,1980,Central Indian,Subantarctic,1,89.864761,531.076147,692.923480,4498.345215
3978,2013-11-12T00:00:00Z,2013-11,2013,11,-57.829,-26.35,-57.803,-26.3,autumn,weaning,2010,Atlantic,Antarctic,1,NaN,1443.071239,1067.076760,506.524384


## Loading environmental data from observations

In [138]:
#Creating dictionary with useful information
varDict = {'var_name': 'dist_ice_edge_km',
           #Folder containing obs
           'obs_main': '/g/data/v45/la6889/Chapter2_Crabeaters/SeaIceObs/Distance_Edge/*.nc',
           #Output folder
           'base_out': '../Cleaned_Data'}

In [141]:
#Getting list of all obs in folder
files_var = sorted(glob(varDict['obs_main']))

#Loading all data into single dataset
var_df = xr.open_mfdataset(files_var)
var_df = var_df.rename_vars({'dist_km': 'dist_ice_edge_km'})
var_df = var_df.dist_ice_edge_km.rename({'lon': 'xt_ocean', 'lat': 'yt_ocean'})

#Checking results
var_df

<xarray.DataArray 'dist_ice_edge_km' (time: 494, yt_ocean: 740, xt_ocean: 3600)>
dask.array<concatenate, shape=(494, 740, 3600), dtype=float64, chunksize=(1, 740, 3600), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1978-11-15 1978-12-15 ... 2019-12-15
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Subsetting variables by time to match observations
By the subsetting the original dataset, we will reduce computing time.

In [142]:
#Getting years and month available in the env data from observations
var_dates = [f'{y}-{str(m).zfill(2)}' for y, m in zip(var_df.time.dt.year.values.tolist(), 
                                                      var_df.time.dt.month.values.tolist())]

#Matching with unique dates when crabeaters where observed
timesteps = sorted([d for d in crabeaters.date.unique() if d in var_dates])

#Creating an empty list to keep the subset model data
var_df_int = []
#Looping through each time
for t in timesteps:
    var_df_int.append(var_df.sel(time = t))
    
#Creating a new data array with the time steps of interest
var_df_int = xr.concat(var_df_int, dim = 'time')
#Dealing original model data
del var_df

#Checking results - Time steps now match mask
var_df_int

<xarray.DataArray 'dist_ice_edge_km' (time: 128, yt_ocean: 740, xt_ocean: 3600)>
dask.array<concatenate, shape=(128, 740, 3600), dtype=float64, chunksize=(1, 740, 3600), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1981-12-15 1982-01-15 ... 2019-12-15
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Applying crabeater observations mask

In [143]:
#Applying mask
var_masked = var_df_int.where(~np.isnan(mask_all), drop = True)

In [144]:
#Converting to pandas data frame
var_pd = var_masked.to_series().dropna().reset_index()

#Adding year and month column prior to merging with crabeater observations
var_pd['year'] = var_pd.apply(lambda i: i.time.year, axis = 1)
var_pd['month'] = var_pd.apply(lambda i: i.time.month, axis = 1)

#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Removing time column that is not needed
var_pd = var_pd.drop(columns = 'time')

#Checking results
var_pd.head()

,yt_ocean,xt_ocean,dist_ice_edge_km,year,month
0,-77.687,166.35,1322.329468,1981,12
1,-76.842,-36.05,1484.290587,1981,12
2,-76.758,-157.25,1257.479413,1981,12
3,-76.758,-156.85,1256.538540,1981,12
4,-76.378,163.95,1172.483253,1981,12


In [145]:
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_name']).columns.tolist()
cols

['yt_ocean', 'xt_ocean', 'year', 'month']

In [146]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,...,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m,SIC,SST_degC,lt_pack_ice,dist_ice_edge_km
0,2004-02-19T00:00:00Z,2004-02,2004,2,-77.687,166.35,-77.708,166.3,summer,moulting,...,Antarctic,1,89.980469,606.013921,32.155391,176.033340,NaN,-1.237460,0.000000,177.309322
1,2014-01-13T14:51:22Z,2014-01,2014,1,-76.842,-36.05,-76.864,-36.1,summer,moulting,...,Antarctic,1,89.729942,383.202891,41.998688,1064.320435,0.885316,-1.623010,0.880952,712.582444
2,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-157.25,-76.779,-157.3,summer,moulting,...,Antarctic,1,89.943176,113.770361,4.724272,612.181458,0.010287,-0.901687,0.654762,862.799495
3,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-156.85,-76.779,-156.8,summer,moulting,...,Antarctic,1,89.897667,113.278688,2.641858,656.099976,0.010458,-0.911558,0.690476,866.721257
4,2020-02-02T00:00:00Z,2020-02,2020,2,-76.378,163.95,-76.357,163.9,summer,moulting,...,Antarctic,1,89.871246,603.310632,21.026712,519.266663,NaN,-1.403092,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,1989-11-05T10:00:00Z,1989-11,1989,11,-60.047,70.25,-60.072,70.2,autumn,weaning,...,Subantarctic,1,89.978432,478.561536,622.374268,4235.086426,0.213187,-1.277349,0.035714,87.512018
3976,2008-12-26T00:00:00Z,2008-12,2008,12,-59.645,9.15,-59.620,9.1,summer,weaning,...,Antarctic,1,89.958000,844.112672,926.684712,5462.944336,0.418786,-1.270739,0.380952,294.557622
3977,1989-11-05T02:00:00Z,1989-11,1989,11,-59.493,71.85,-59.468,71.9,autumn,weaning,...,Subantarctic,1,89.864761,531.076147,692.923480,4498.345215,0.155776,-1.218217,0.000000,33.037999
3978,2013-11-12T00:00:00Z,2013-11,2013,11,-57.829,-26.35,-57.803,-26.3,autumn,weaning,...,Antarctic,1,NaN,1443.071239,1067.076760,506.524384,0.155182,-1.245528,0.000000,11.858549


In [147]:
crabeaters.to_csv('../Cleaned_Data/Env_obs/unique_crabeater_obs_all_env.csv', index = False)